In [1]:
from flask import Flask, request, jsonify
from threading import Thread

import json

In [2]:
!pkill -f flask


In [3]:
from flask import Flask, request, jsonify
from threading import Thread
import json
import os
from datetime import datetime, timedelta
from dateutil import parser as dt_parser
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
import requests

In [4]:
app = Flask(__name__)
received_data = []

In [5]:
def retrieve_calendar_events(user, start, end):
    token_path = os.path.join("../Keys", f"{user.split('@')[0]}.token")
    creds = Credentials.from_authorized_user_file(token_path)
    service = build("calendar", "v3", credentials=creds)
    events = service.events().list(
        calendarId=user,
        timeMin=start,
        timeMax=end,
        singleEvents=True,
        orderBy='startTime'
    ).execute().get('items', [])
    result = []
    for ev in events:
        s = ev['start'].get('dateTime') or ev['start'].get('date') + "T00:00:00Z"
        e = ev['end'].get('dateTime') or ev['end'].get('date') + "T00:00:00Z"
        atts = [a.get('email') for a in ev.get('attendees', [])] or [user]
        result.append({
            "StartTime": s,
            "EndTime": e,
            "NumAttendees": len(atts),
            "Attendees": atts,
            "Summary": ev.get('summary', '')
        })
    return result

In [6]:
def parse_email(email_text):
    payload = {
        "model": "meta-llama",
        "messages": [
            {"role": "user", "content": f"""
You are an autonomous meeting scheduler. Extract and return JSON:
- participants: list of emails
- meeting_duration: integer minutes
- earliest: ISO datetime start window
- latest: ISO datetime end window
If names only, append @amd.com. No extra text.
Email: {email_text}
"""}
        ]
    }
    res = requests.post(LLM_URL, json=payload)
    return json.loads(res.json()['choices'][0]['message']['content'])

In [7]:
def find_meeting_slot(busy, duration, earliest, latest):
    start_window = dt_parser.isoparse(earliest)
    end_window = dt_parser.isoparse(latest)
    delta = timedelta(minutes=duration)
    step = timedelta(minutes=TIME_STEP_MINUTES)


In [8]:
def find_meeting_slot(busy, duration, earliest, latest):
    start_window = dt_parser.isoparse(earliest)
    end_window = dt_parser.isoparse(latest)
    delta = timedelta(minutes=duration)
    step = timedelta(minutes=TIME_STEP_MINUTES)

    # search across multiple days if needed
    for day_offset in range(MAX_SEARCH_DAYS):
        day_start = start_window + timedelta(days=day_offset)
        window_start = day_start
        window_end = end_window + timedelta(days=day_offset)
        t = window_start
        while t + delta <= window_end:
            conflict = False
            for slots in busy.values():
                for slot in slots:
                    s = dt_parser.isoparse(slot['StartTime'])
                    e = dt_parser.isoparse(slot['EndTime'])
                    if not (t + delta <= s or t >= e):
                        conflict = True
                        break
                if conflict:
                    break
            if not conflict:
                return t.isoformat(), (t + delta).isoformat()
            t += step
    return None, None

In [9]:
from calendar_utils import retrieve_calendar_events


In [10]:
def your_meeting_assistant(data):
    content = data['EmailContent']
    subject = data.get('Subject', 'Meeting')
    sender = data['From']
    base_time = data['Datetime']

    # 1) Parse email for scheduling info
    parsed = parse_email(content)
    participants = parsed['participants']
    duration = parsed['meeting_duration']
    earliest = parsed['earliest']
    latest = parsed['latest']

    # ensure sender included
    all_users = list({sender} | set(participants))

    # 2) Fetch each user's busy events
    busy_events = {}
    for user in all_users:
        busy_events[user] = retrieve_calendar_events(
            user, earliest, latest)

    # 3) Find a conflict-free slot
    start, end = find_meeting_slot(busy_events, duration, earliest, latest)

    # 4) Build attendee schedule output
    attendee_objs = []
    for user in all_users:
        evs = busy_events[user]
        if start and end:
            evs.append({
                'StartTime': start,
                'EndTime': end,
                'NumAttendees': len(all_users),
                'Attendees': all_users,
                'Summary': subject
            })
        attendee_objs.append({'email': user, 'events': evs})

    # 5) Construct final response
    data['Attendees'] = attendee_objs
    data['EventStart'] = start or ''
    data['EventEnd'] = end or ''
    data['Duration_mins'] = str(duration) if start else ''
    data['processed'] = parsed
    if start:
        data['output'] = {'status': 'scheduled', 'Link': None}
    else:
        fallback = (dt_parser.isoparse(earliest) + timedelta(days=1)).isoformat()
        data['output'] = {'status': 'conflict', 'suggested': fallback}

    return data

In [16]:
if not app.view_functions.get('receive'):
    @app.route('/receive', methods=['POST'])
    def receive():
        data = request.get_json(force=True)
        result = your_meeting_assistant(data)
        received_data.append(result)
        return jsonify(result)


In [17]:
def run_flask():
    app.run(host='0.0.0.0', port=5050, debug=False)

Thread(target=run_flask, daemon=True).start()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5050
 * Running on http://134.199.205.253:5050
INFO:werkzeug:Press CTRL+C to quit


In [18]:
# Start Flask in a background thread
Thread(target=run_flask, daemon=True).start()

 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5050 is in use by another program. Either identify and stop that program, or start the server with a different port.
